In [1]:
#Set Dependencies

%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [94]:
# File to Load (Remember to change these)
#Solar Installations from https://openpv.nrel.gov/search
#Climate Zone Data is from California Energy Commission. California Climate Zone Descriptions for New Buildings. July 1995.
#Climate Zone Details is from California Energy Commission. California Climate Zone Descriptions for New Buildings. July 1995.
#Sunny Days by https://www.currentresults.com/Weather/US/average-annual-sunshine-by-city.php included in Climate Zone csv
#Population data from
#Electricy Consumption by county from http://ecdms.energy.ca.gov/elecbycounty.aspx


#Read files Data from download
pv_data_df = pd.read_csv("Data/open_pv_california_2015.csv", encoding="ISO-8859-1")
climate_zip_code =pd.read_csv("Data/BuildingClimateZonesByZIPCode.csv", encoding="ISO-8859-1")
population = pd.read_csv("Data/population.csv",encoding="ISO-8859-1" )
electricity_consumption=pd.read_csv("Data/ElectricityConsumptionByCounty.csv",encoding="ISO-8859-1" )
climate_zone_details=pd.read_csv("Data/ClimateZoneDetails.csv",encoding="ISO-8859-1" )



C:\Users\aruba\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#SOLAR PANEL DATA

#Clean data by only selecting the columns we need and filter by residential

solar_data = pv_data_df[["incentive_prog_names","size_kw","zipcode","install_type",
                         "cost_per_watt","utility_clean","county","annual_PV_prod"]]
solar_data=solar_data.dropna()

solar_data.loc[solar_data["install_type"] == "residential"]

solar_data=solar_data[["incentive_prog_names","size_kw","zipcode",
                         "cost_per_watt","utility_clean","county","annual_PV_prod"]]


In [7]:
#MERGE IN THE CLIMATE ZONE BY ZIP CODE, POPULATION AND ELECTRICITY CONSUMPTION

#Merge Solar Data with Climate Zone zip code
solar_cz_data = pd.merge(solar_data, climate_zip_code, on="zipcode")

#Merge in population and electricity consumption
solar_cz_pop = pd.merge(solar_cz_data, population, on="county")
solar_cz_pop_elec = pd.merge(solar_cz_pop, electricity_consumption, on="county" )

#MAIN DATA BASE WITH ALL INFORMATION NEEDED TO START ANALYSIS
solar_summary=solar_cz_pop_elec.rename(columns = {"incentive_prog_names": "Incentive Program", 
                                                      "size_kw": "System Size kw", 
                                                      "cost_per_watt": "Cost Per Watt",
                                                     "utility_clean": "Utility Company",
                                                      "county": "County", 
                                                      "annual_PV_prod": "Solar Energy Production kwh",
                                                      "population": "Population",
                                                      "housing_units": "Housing Units",
                                                      "2015 Millions of kWh (GWh)": "Electricity Production Millions of kWh (GWh)"
                                                     })
# Save as a csv
solar_summary.to_csv("solar_summary.csv", encoding="utf-8", index=False)
solar_summary.head()


Incentive Program  System Size kw  zipcode  \
0  California Public Utilities Commission (Non-CS...          7.0672    93720   
1  California Public Utilities Commission (Non-CS...          6.5604    93720   
2  California Public Utilities Commission (Non-CS...          6.5604    93720   
3  California Public Utilities Commission (Non-CS...          9.9320    93720   
4  California Public Utilities Commission (Non-CS...          8.2005    93720   

   Cost Per Watt                 Utility Company  County  \
0       3.162497  Pacific Gas & Electric Company  Fresno   
1       1.704481  Pacific Gas & Electric Company  Fresno   
2       1.704481  Pacific Gas & Electric Company  Fresno   
3       4.063711  Pacific Gas & Electric Company  Fresno   
4       4.208194  Pacific Gas & Electric Company  Fresno   

   Solar Energy Production kwh  Building CZ  Population  Housing Units  \
0                 11080.851980           13      930450         315531   
1                  9368.967142           13      930450         315531   
2                  9008.908333           13      930450         315531   
3                 14009.671590           13      930450         315531   
4                 10891.724510           13      930450         315531   

   land_area_sq_mi  Electricity Production Millions of kWh (GWh)  
0             5958                                   2674.195219  
1             5958                                   2674.195219  
2             5958                                   2674.195219  
3             5958                                   2674.195219  
4             5958                                   2674.195219

In [44]:
#SHOW DATA GROUPBY CLIMATE ZONE AND MERGE IN THE ADDITIONAL INFORMATION FROM 


cz_count=pd.DataFrame(solar_summary["Building CZ"].value_counts()).reset_index()
cz_count=cz_count.rename(columns = {"index": "Climate Location Zone","Building CZ": "PV Installations"})
cz_count=cz_count.sort_values("Climate Location Zone").set_index("Climate Location Zone")

#Making a new database to insert all information
solardata_summary=cz_count

solardata_summary["System Size kw"]=round(solar_summary.groupby("Building CZ")["System Size kw"].mean(),2)
solardata_summary["Solar Energy Production kwh"]=round(solar_summary.groupby("Building CZ")["Solar Energy Production kwh"].sum())
solardata_summary["Cost Per Watt"]=round(solar_summary.groupby("Building CZ")["Cost Per Watt"].mean(),2)
solardata_summary["Population Size"]=round(solar_summary.groupby("Building CZ")["Population"].mean())
solardata_summary["Housing Units"]=round(solar_summary.groupby("Building CZ")["Housing Units"].mean())
solardata_summary["Ratio of Intallations:Housing Units"]=round(100*solardata_summary["PV Installations"]/solardata_summary["Housing Units"], 2)
solardata_summary["Electricity (Not Solar) Production Millions of kWh (GWh)"]=round(solar_summary.groupby("Building CZ")["Electricity Production Millions of kWh (GWh)"].sum(),2)

solardata_summary=pd.DataFrame(solardata_summary)
solardata_summary

PV Installations  System Size kw  \
Climate Location Zone                                     
1                                   108            5.20   
2                                  1640            7.00   
3                                  3195            7.28   
4                                  4958            6.26   
5                                  1501            6.54   
6                                  5501            7.22   
7                                   662            6.75   
8                                  8447            6.30   
9                                  8395            6.86   
10                                 7178            6.87   
11                                 3970            7.38   
12                                11697            6.93   
13                                 9412           11.46   
14                                 2306            5.92   
15                                    8            8.48   
16                                  630            6.40   

                       Solar Energy Production kwh  Cost Per Watt  \
Climate Location Zone                                               
1                                         726761.0           4.73   
2                                       16603626.0           5.12   
3                                       34108696.0           5.08   
4                                       46239079.0           4.82   
5                                       15733500.0           4.88   
6                                       58172967.0           4.78   
7                                        6807486.0           4.68   
8                                       79851034.0           4.84   
9                                       86698586.0           4.89   
10                                      74235982.0           4.51   
11                                      41741882.0           4.66   
12                                     116020716.0           4.95   
13                                     163508675.0           4.78   
14                                      20817803.0           4.81   
15                                        114639.0           4.43   
16                                       5959626.0           4.80   

                       Population Size  Housing Units  \
Climate Location Zone                                   
1                             209001.0        92015.0   
2                             346640.0       147471.0   
3                             970061.0       374428.0   
4                            1451850.0       517860.0   
5                             341473.0       133856.0   
6                            4685110.0      1639573.0   
7                            3095313.0      1164786.0   
8                            5419311.0      1896784.0   
9                            8381716.0      2939764.0   
10                           2403494.0       883265.0   
11                            216918.0        93873.0   
12                            856912.0       323519.0   
13                            702601.0       235666.0   
14                           7974705.0      2796639.0   
15                           2189641.0       800707.0   
16                           3528211.0      1236339.0   

                       Ratio of Intallations:Housing Units  \
Climate Location Zone                                        
1                                                     0.12   
2                                                     1.11   
3                                                     0.85   
4                                                     0.96   
5                                                     1.12   
6                                                     0.34   
7                                                     0.06   
8                                                     0.45   
9                                                     0.29   
10      

In [54]:
#CLIMATE ZONES/lOCATION ZONES DETAILS DATA
climate_zone_details=climate_zone_details.rename(columns = {"Climate/Location Zone": "Climate Location Zone", 
                                                      "reference_city": "Reference City", 
                                                      "cz_location": "Climate Zone Ref City",
                                                      "sunny_days": "Sunny Days",
                                                      "highest_temp_rec": "Highest Temp Rec", 
                                                      "lowest_temp_rec": "Lowest Temp Rec",
                                                      "Latitude": "Latitude",
                                                      "Longitude": "Longitude",
                                                      "Elevation": "Elevation",
                                                      "HDd": "Heating Degree Days", 
                                                      "CDD": "Cooling Degree Days",
                                                     })
climate_zone_details

Climate Location Zone            Reference City  \
0                       1                    Eureka   
1                       2                      Napa   
2                       3                   Oakland   
3                       4                  San Jose   
4                       5               Santa Maria   
5                       6               Los Angeles   
6                       7                 San Diego   
7                       8                Long Beach   
8                       9  Los Angeles Civic Center   
9                      10                 Riverside   
10                     11                 Red Bluff   
11                     12                  Stockton   
12                     13                    Fresno   
13                     14                   Barstow   
14                     15                   Brawley   
15                     16                    Bishop   

         Climate Zone Ref City  Sunny Days  Highest Temp Rec  Lowest Temp Rec  \
0                     1-Eureka         179                85               21   
1                       2-Napa         260               113               14   
2                    3-Oakland         261               113               14   
3                   4-San Jose         257               109               19   
4                5-Santa Maria         283               108               20   
5                6-Los Angeles         284               110               27   
6                  7-San Diego         263               111               29   
7                 8-Long Beach         278               111               25   
8   9-Los Angeles Civic Center         263               110               28   
9                 10-Riverside         277               116               19   
10                11-Red Bluff         249               119               20   
11                 12-Stockton         257               114               19   
12                   13-Fresno         271               111               19   
13                  14-Barstow         281               116                3   
14                  15-Brawley         305               122                2   
15                   16-Bishop         296               109               -7   

    Latitude  Longitude  Elevation  Heating Degree Days  Cooling Degree Days  
0      41.30     124.28         43                 4496                    0  
1      38.28     122.27         60                 2844                  456  
2      37.75     122.20         10                 2909                  128  
3      37.35     121.90         70                 2335                  574  
4      34.93     120.42        230                 2844                  456  
5      33.93     118.40        110                 1902                  470  
6      32.73     117.17         10                 2009                  505  
7      33.82     118.15         30                 1430                 1201  
8      34.05     118.23        270                 1154                 1537  
9      33.95     117.38        840                 1904                 1714  
10     40.09     122.15        342                 2688                 1904  
11     37.54     121.15         22                 2702                 1470  
12     36.46     119.43        328                 2702                 1470  
13     35.00     116.47       1927                 2581                 4239  
14     32.95     115.55          0                 1106                 6565  
15     37.22     118.22       4108                 4313                 1037

In [67]:
#MERGE DATA FROM SOLAR DATA SUMMARY AND CLIMATE LOCATION ZONE DETAILS

summary_by_climatelocation=pd.merge(solardata_summary, climate_zone_details, on="Climate Location Zone")
summary_by_climatelocation

Climate Location Zone  PV Installations  System Size kw  \
0                       1               108            5.20   
1                       2              1640            7.00   
2                       3              3195            7.28   
3                       4              4958            6.26   
4                       5              1501            6.54   
5                       6              5501            7.22   
6                       7               662            6.75   
7                       8              8447            6.30   
8                       9              8395            6.86   
9                      10              7178            6.87   
10                     11              3970            7.38   
11                     12             11697            6.93   
12                     13              9412           11.46   
13                     14              2306            5.92   
14                     15                 8            8.48   
15                     16               630            6.40   

    Solar Energy Production kwh  Cost Per Watt  Population Size  \
0                      726761.0           4.73         209001.0   
1                    16603626.0           5.12         346640.0   
2                    34108696.0           5.08         970061.0   
3                    46239079.0           4.82        1451850.0   
4                    15733500.0           4.88         341473.0   
5                    58172967.0           4.78        4685110.0   
6                     6807486.0           4.68        3095313.0   
7                    79851034.0           4.84        5419311.0   
8                    86698586.0           4.89        8381716.0   
9                    74235982.0           4.51        2403494.0   
10                   41741882.0           4.66         216918.0   
11                  116020716.0           4.95         856912.0   
12                  163508675.0           4.78         702601.0   
13                   20817803.0           4.81        7974705.0   
14                     114639.0           4.43        2189641.0   
15                    5959626.0           4.80        3528211.0   

    Housing Units  Ratio of Intallations:Housing Units  \
0         92015.0                                 0.12   
1        147471.0                                 1.11   
2        374428.0                                 0.85   
3        517860.0                                 0.96   
4        133856.0                                 1.12   
5       1639573.0                                 0.34   
6       1164786.0                                 0.06   
7       1896784.0                                 0.45   
8       2939764.0                                 0.29   
9        883265.0                                 0.81   
10        93873.0                                 4.23   
11       323519.0                                 3.62   
12       235666.0                                 3.99   
13      2796639.0                                 0.08   
14       800707.0                                 0.00   
15      1236339.0                                 0.05   

    Electricity (Not Solar) Production Millions of kWh (GWh)  \
0                                        6.267468e+04          
1                                        1.512592e+06          
2                                        6.335256e+06          
3                                        1.562035e+07          
4                                        1.044654e+06          
5                                        5.526495e+07          
6                                        4.556752e+06          
7                                        9.871762e+07          
8                                        1.468504e+08          
9                                        4.917980e+07          
10                                       3.168247e+06          
11                                   

In [ ]:
#GRAPH 1 GEOMAP SHOWING THE LOCATION OF CLIMATE ZONES

In [ ]:
#1 HYPOTHESIS: THE LOCATION OF CLIMATE ZONES 11, 12, 13 WILL HAVE MORE SOLAR INSTALLATIONS

#GRAPH 2 SCATTER PLOT SHOWING THE AMOUNT OF INSTALLATIONS BY CLIMATE ZONE
#GRAPH 3 HEAT MAP WITH THE INSTALLATIONS BY CLIMATE/LOCATION ZONE


In [ ]:
#2 HYPOTHESIS:  CLIMATE ZONES WITH THE HIGHEST TEMPERATURE WILL HAVE MORE SOLAR INSTALLATIONS
#GRAPH 4:  CLIMATE ZONES TEMPERATURES VS. INSTALLATIONS 
#STATISTIC TEST HERE


In [ ]:
#3 HYPOTHESIS:  THE LARGER THE POPULATION THE LARGER THE SOLAR INSTALLATIONS
#STATISTIC TEST HERE

In [ ]:
#4 HYPOTHESIS:  THE CLIMATE ZONES WITH MORE INCENTIVE PROGRAMS WILL HAVE MORE INSTALLATIONS
#GRAPH 5:  SHOW NUMBER OF INSTALLATIONS BY INCENTIVE PROGRAM


In [114]:
#DATAFRAME ON CALCULATIONS ON GREENHOUSE EMMISSIONS AND THE IMPACT
#Calculating greenhouse emmissions Electricity: (VARIABLE) kWh per home × 998 lbs CO2 per megawatt-hour generated × (1/(1-0.071)) MWh generated/MWh delivered × 1 MWh/1,000 kWh × 1 metric ton/2,204.6 lb = 5.734 metric tons CO2/home

co2_comparison=summary_by_climatelocation.drop(["System Size kw", "Cost Per Watt", "Climate Location Zone", 
                                                "Housing Units", "Ratio of Intallations:Housing Units", "Reference City", 
                                                "Sunny Days", "Highest Temp Rec", "Lowest Temp Rec", "Latitude", "Longitude", 
                                                "Elevation", "Heating Degree Days", "Cooling Degree Days", "Population Size",
                                                "Electricity (Not Solar) Production Millions of kWh (GWh)" ], axis=1)

co2_comparison=co2_comparison[["Climate Zone Ref City","PV Installations","Solar Energy Production kwh" ]]
co2_comparison

Climate Zone Ref City  PV Installations  Solar Energy Production kwh
0                     1-Eureka               108                     726761.0
1                       2-Napa              1640                   16603626.0
2                    3-Oakland              3195                   34108696.0
3                   4-San Jose              4958                   46239079.0
4                5-Santa Maria              1501                   15733500.0
5                6-Los Angeles              5501                   58172967.0
6                  7-San Diego               662                    6807486.0
7                 8-Long Beach              8447                   79851034.0
8   9-Los Angeles Civic Center              8395                   86698586.0
9                 10-Riverside              7178                   74235982.0
10                11-Red Bluff              3970                   41741882.0
11                 12-Stockton             11697                  116020716.0
12                   13-Fresno              9412                  163508675.0
13                  14-Barstow              2306                   20817803.0
14                  15-Brawley                 8                     114639.0
15                   16-Bishop               630                    5959626.0

In [115]:
#AMOUNT OF GREENHOUSE EMISSIONS
greenhousefactor=0.000487287
treefactor=
est_greenEmissions_ifnot_solar=[]
trees_needed_per_kw=[]


